# Balanceo de conjuntos de evaluación

* **Autor**: Julián Zuloaga
* **Asignatura:** Memoria de Título
---



**Descripción:** En este notebook se genera un algoritmo que balancea el dataset de validación de AffectNet para generar conjuntos de validación y testeo de Custom Mosaics balanceados.

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
#from js2py import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline
# Se importan librerías para procesamiento de imágenes
from PIL import Image
import ntpath
import glob
# Se importan librerías
from IPython.display import clear_output # limpia salida
import os # manejo de directorios
import random # para funciones de randomizado
from PIL import Image, ImageDraw # para manejo de imágenes
import numpy as np # manejo de arrays de numpy
from google.colab.patches import cv2_imshow # para mostrar imágen generada
import cv2 # para visualización y agregar texto


In [ ]:
# Se monta unidad de google drive a la máquina virtual de Colab
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Se crea un symbolic link para reemplazar "/content/gdrive/My\ Drive/" a "/mydrive"
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# Se crea directorio local para guardar el dataset de validación de AffectNet
!mkdir /content/output_data/

In [ ]:
# Se copia base de datos de validación a directorio local
!cp /mydrive/Proc_AffectNet_YOLOv4_v1/test.zip /content/output_data/test.zip 

In [ ]:
# se descomprime el dataset
%cd /content/output_data/
!unzip ./test.zip -d ./test_completo/

In [ ]:
# Se procede a generar lista con el nombre de las imágenes del set de validación completo
img_list = [x for x in glob.glob("/content/output_data/test_completo/*.jpg")]

In [ ]:
# Se define lista con categorías de emoción de la base de datos
class_list = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger','Contempt']

In [ ]:
# Se identifican las imágenes de cada categoría y se separa su directorio en listas distintas
sorted_img_list = dict() # se inicializa diccionario con direcciones de imágenes ordenadas por clase
# se agregan las keys al diccionario y se inicializa lista como values
for j in class_list:
  sorted_img_list[j] = list()
# Se itera la lista de imágenes
for img_addr in img_list:
  # se abre archivo de texto para la imagen correspondiente
  f = open(f"{img_addr[:-4]}.txt", "r")
  # Se lee la línea de parámetros en formato String
  line = f.readline()
  # Se separan los elementos del string
  params = line.split(" ")
  # Se asigna la clase a la variable img_class por medio del parámetro de clase
  img_class = class_list[int(params[0])]
  # una vez identificada la clase de la imagen, se guarda su dirección y clase al diccionario sorted_img_list
  #sorted_img_list[img_class] = img_addr
  sorted_img_list[img_class].append(img_addr)

  # Se cierra el archivo de texto
  f.close()

In [ ]:
# Se verifica contenido del diccionario
elem_counter = 0
for key, value in sorted_img_list.items():
  print(key, value)
  elem_counter += len(value)
print(f'N° de elementos total diccionario: {elem_counter}')

In [ ]:
# Es imporante tener en cuenta la cantidad de imágenes que hay en el set de entrenamiento de AffectNet
# Neutral:  74.874  (26,0%)
# Happy:    134.415 (46,7%)
# Sad:      25.459  (8,85%)
# Surprise: 14.090  (4,89%)
# Fear:     6.378   (2,21%)
# Disgust:  3.803   (1,32%)
# Anger:    24.882  (8,65%)
# Contempt: 3.750   (1,30%)
# -------------------------------
# TOTAL: 287.651 IMÁGENES
#
#
# Para que el conjunto de validación/testeo posea el mismo balance, debería tener
# Neutral:  4.992 (26,03%) 
# Happy:    8.961 (46,7%)
# Sad:      1.697 (8,85%)
# Surprise: 939   (4,89%)
# Fear:     425   (2,21%)
# Disgust:  254   (1,32%)
# Anger:    1.659 (8,65%)
# Contempt: 250   (1,30%)
# ------------------------------
# TOTAL: 19.177 IMÁGENES

In [ ]:
# Tecnicas de Data Augmentation:
# -----------------------------
# a. Horizontal Flip
# b. Random Contrast
# c. Random Gamma
# d. Random Brighten
# e. Linear Blur Filter | este quizás no sea necesario si se considera el original también...

In [ ]:
# Luego, se propone realizar la siguiente lista de técnicas de Data Augmentation
# Neutral:  250 x 5 (Orig+DA) x 4 (mos) = 5.000  | ~4.992 
# Happy:    250 x 5 (Orig+DA) x 6 (mos) = 8.750  | ~8.961
# Sad:      250 x 3 (Orig+DA) x 3 (mos) = 1.500  | ~1.697 
# Surprise: 250 x 2 (Orig+DA) x 2 (mos) = 1000   | ~939
# Fear:     250 x 2 (mos) = 500                  | ~425
# Disgust:  250 x 1 (mos) = 250                  | ~254
# Anger:    250 x 2 (Orig+DA) x 3 (mos)          | ~1.500
# Contempt: 250 (Original)

In [ ]:
# se crean directorios para los datasets de salida (validación y testeo)
!mkdir /content/output_sets/ /content/output_sets/val /content/output_sets/test

In [ ]:
import os # manejo de directorios
# Se crean directorios con las clases
val_addr = "/content/output_sets/val/"
test_addr = "/content/output_sets/test/"
for class_name in class_list:
  os.mkdir(f"{val_addr}{class_name}/")
  os.mkdir(f"{test_addr}{class_name}/")

# Procesamiento de dataset

## Clase Neutral

In [ ]:
from PIL import Image, ImageEnhance # se importan librerías para procesamiento de imágenes
import shutil # para copiar archivos de texto

# Se define contador de imágenes
img_counter = 0
# Se define clase
img_class = "Neutral"
# Se procesa el conjunto Neutral:
for img_addr in sorted_img_list.get(img_class):
  # se abre imagen
  img = Image.open(img_addr)
  # se aplica horizontal flip
  img_horizontal_flip = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
  # se aplica cambio de contraste
  enhancer = ImageEnhance.Contrast(img)   # se define enhancer para contraste
  img_contrast = enhancer.enhance(1.5)    # se aumenta el contraste
  # se aplica cambio de brillo
  enhancer = ImageEnhance.Brightness(img) # se define enhancer para brillo
  img_brightness = enhancer.enhance(1.5)  
  # Se aplica cambio de saturación
  enhancer = ImageEnhance.Color(img)      # se define enhancer para saturación
  img_saturation = enhancer.enhance(0.5)
  # Se determina directorio de salida en base al numero de imágenes procesadas para la clase
  img_counter += 1
  if img_counter > 250:
    output_dir = "/content/output_sets/val/"
  else:
    output_dir = "/content/output_sets/test/"
  # se guardan las imágenes
  img.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.jpg")
  img_horizontal_flip.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_horizontal_flip.jpg")
  img_contrast.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_contrast.jpg")
  img_brightness.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_brightness.jpg")
  img_saturation.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_saturation.jpg")
  # se guardan las anotaciones para dichas imágenes
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_horizontal_flip.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_contrast.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_brightness.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_saturation.txt")
  # Se cierran las imágenes
  img.close()
  img_horizontal_flip.close()
  img_contrast.close()
  img_brightness.close()
  img_saturation.close()

## Clase Happy

In [ ]:
# Se define contador de imágenes
img_counter = 0
# Se define clase
img_class = "Happy"
# Se procesa el conjunto Happy:
for img_addr in sorted_img_list.get(img_class):
  # se abre imagen
  img = Image.open(img_addr)
  # se aplica horizontal flip
  img_horizontal_flip = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
  # se aplica cambio de contraste
  enhancer = ImageEnhance.Contrast(img)   # se define enhancer para contraste
  img_contrast = enhancer.enhance(1.5)    # se aumenta el contraste
  # se aplica cambio de brillo
  enhancer = ImageEnhance.Brightness(img) # se define enhancer para brillo
  img_brightness = enhancer.enhance(1.5)  
  # Se aplica cambio de saturación
  enhancer = ImageEnhance.Color(img)      # se define enhancer para saturación
  img_saturation = enhancer.enhance(0.5)
  # Se aplica filtro de desenfoque
  enhancer = ImageEnhance.Sharpness(img) # se define enhancer para desenfoque (Linear Box Filter)
  img_blur = enhancer.enhance(0.1)
  enhancer = ImageEnhance.Sharpness(img_blur) # se aplica filtro de desenfoque por 2a vez
  img_blur = enhancer.enhance(0.1)
  enhancer = ImageEnhance.Sharpness(img_blur) # se aplica filtro de desenfoque por 3a vez
  img_blur = enhancer.enhance(0.1)
  # Se determina directorio de salida en base al numero de imágenes procesadas para la clase
  img_counter += 1
  if img_counter > 250:
    output_dir = "/content/output_sets/val/"
  else:
    output_dir = "/content/output_sets/test/"
  # se guardan las imágenes
  img.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.jpg")
  img_horizontal_flip.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_horizontal_flip.jpg")
  img_contrast.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_contrast.jpg")
  img_brightness.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_brightness.jpg")
  img_saturation.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_saturation.jpg")
  img_blur.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_blur.jpg")
  # se guardan las anotaciones para dichas imágenes
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_horizontal_flip.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_contrast.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_brightness.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_saturation.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_blur.txt")
  # Se cierran las imágenes
  img.close()
  img_horizontal_flip.close()
  img_contrast.close()
  img_brightness.close()
  img_saturation.close()
  img_blur.close()

## Clase Sad

In [ ]:
from PIL import Image, ImageEnhance # se importan librerías para procesamiento de imágenes
# Se define contador de imágenes
img_counter = 0
# Se define clase
img_class = "Sad"
# Se procesa el conjunto Sad:
for img_addr in sorted_img_list.get(img_class):
  # se abre imagen
  img = Image.open(img_addr)
  # se aplica horizontal flip
  img_horizontal_flip = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
  # Se determina directorio de salida en base al numero de imágenes procesadas para la clase
  img_counter += 1
  if img_counter > 250:
    output_dir = "/content/output_sets/val/"
  else:
    output_dir = "/content/output_sets/test/"
  # se guardan las imágenes
  img.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.jpg")
  img_horizontal_flip.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_horizontal_flip.jpg")
  # se guardan las anotaciones para dichas imágenes
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_horizontal_flip.txt")
  # Se cierran las imágenes
  img.close()
  img_horizontal_flip.close()

## Clase Surprise

In [ ]:
from PIL import Image, ImageEnhance # se importan librerías para procesamiento de imágenes
# Se define contador de imágenes
img_counter = 0
# Se define clase
img_class = "Surprise"
# Se procesa el conjunto Surprise:
for img_addr in sorted_img_list.get(img_class):
  # se abre imagen
  img = Image.open(img_addr)
  # se aplica horizontal flip
  img_horizontal_flip = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
  # Se determina directorio de salida en base al numero de imágenes procesadas para la clase
  img_counter += 1
  if img_counter > 250:
    output_dir = "/content/output_sets/val/"
  else:
    output_dir = "/content/output_sets/test/"
  # se guardan las imágenes
  img.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.jpg")
  img_horizontal_flip.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_horizontal_flip.jpg")
    # se guardan las anotaciones para dichas imágenes
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_horizontal_flip.txt")
  # Se cierran las imágenes
  img.close()
  img_horizontal_flip.close()

## Clase Fear

In [ ]:
from PIL import Image, ImageEnhance # se importan librerías para procesamiento de imágenes
# Se define contador de imágenes
img_counter = 0
# Se define clase
img_class = "Fear"
# Se procesa el conjunto Fear:
for img_addr in sorted_img_list.get(img_class):
  # se abre imagen
  img = Image.open(img_addr)
  # Se determina directorio de salida en base al numero de imágenes procesadas para la clase
  img_counter += 1
  if img_counter > 250:
    output_dir = "/content/output_sets/val/"
  else:
    output_dir = "/content/output_sets/test/"
  # se guardan las imágenes
  img.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.jpg")
  # se guardan las anotaciones para dichas imágenes
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.txt")
  # Se cierran las imágenes
  img.close()

## Clase Disgust

In [ ]:
from PIL import Image, ImageEnhance # se importan librerías para procesamiento de imágenes
# Se define contador de imágenes
img_counter = 0
# Se define clase
img_class = "Disgust"
# Se procesa el conjunto Disgust:
for img_addr in sorted_img_list.get(img_class):
  # se abre imagen
  img = Image.open(img_addr)
  # Se determina directorio de salida en base al numero de imágenes procesadas para la clase
  img_counter += 1
  if img_counter > 250:
    output_dir = "/content/output_sets/val/"
  else:
    output_dir = "/content/output_sets/test/"
  # se guardan las imágenes
  img.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.jpg")
  # se guardan las anotaciones para dichas imágenes
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.txt")
  # Se cierran las imágenes
  img.close()

## Clase Anger

In [ ]:
# Se define contador de imágenes
img_counter = 0
# Se define clase
img_class = "Anger"
# Se procesa el conjunto Anger:
for img_addr in sorted_img_list.get(img_class):
  # se abre imagen
  img = Image.open(img_addr)
  # se aplica horizontal flip
  img_horizontal_flip = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
  # se aplica cambio de contraste
  img_counter += 1
  if img_counter > 250:
    output_dir = "/content/output_sets/val/"
  else:
    output_dir = "/content/output_sets/test/"
  # se guardan las imágenes
  img.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.jpg")
  img_horizontal_flip.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_horizontal_flip.jpg")
  # se guardan las anotaciones para dichas imágenes
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.txt")
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_horizontal_flip.txt")
  # Se cierran las imágenes
  img.close()
  img_horizontal_flip.close()

## Clase Contempt

In [ ]:
# Se define contador de imágenes
img_counter = 0
# Se define clase
img_class = "Contempt"
# Se procesa el conjunto Contempt:
for img_addr in sorted_img_list.get(img_class):
  # se abre imagen
  img = Image.open(img_addr)
  # Se determina directorio de salida en base al numero de imágenes procesadas para la clase
  img_counter += 1
  if img_counter > 250:
    output_dir = "/content/output_sets/val/"
  else:
    output_dir = "/content/output_sets/test/"
  # se guardan las imágenes
  img.save(f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.jpg")
  # se guardan las anotaciones para dichas imágenes
  shutil.copyfile(f"{img_addr[:-4]}.txt", f"{output_dir}{img_class}/{os.path.basename(img_addr)[:-4]}_normal.txt")
  # Se cierran las imágenes
  img.close()

# Guardado de los datasets

In [ ]:
# Primero, se verifica el número de imágenes generadas para cada categoría, de cada conjunto

# Set de validación
print("---------------- SET DE VALIDACIÓN ----------------")
img_counter = 0
for img_class in class_list:
  img_list = [x for x in glob.glob(f"/content/output_sets/val/{img_class}/*.jpg")]
  print(f"Clase {img_class}: {len(img_list)}")
  img_counter = img_counter + len(img_list)

print(f"Imágenes totales encontradas: {img_counter}")

# Set de testeo
print("---------------- SET DE TESTEO ----------------")
img_counter = 0
for img_class in class_list:
  img_list = [x for x in glob.glob(f"/content/output_sets/test/{img_class}/*.jpg")]
  print(f"Clase {img_class}: {len(img_list)}")
  img_counter = img_counter + len(img_list)

print(f"Imágenes totales encontradas: {img_counter}")

In [ ]:
# Ahora, se procede a copiar todos los archivos a carpetas para guardarlas en formato YOLOv4
!mkdir /content/output_sets/val/test /content/output_sets/test/test # carpetas finales

In [ ]:
# Validación
# Se llena lista con direcciones de todos los archivos del set de validación
total_file_list = list()
for file_class in class_list:
  file_list = [x for x in glob.glob(f"/content/output_sets/val/{file_class}/*.*")]
  for elem in file_list:
    total_file_list.append(elem)
# Se copian los archivos de la lista al directorio final (carpeta "test")
for file_addr in total_file_list:
  shutil.copyfile(file_addr, f"/content/output_sets/val/test/{os.path.basename(file_addr)}")
total_file_list.clear()

# Testeo
# Se llena lista con direcciones de todos los archivos del set de testeo
total_file_list = list()
for file_class in class_list:
  file_list = [x for x in glob.glob(f"/content/output_sets/test/{file_class}/*.*")]
  for elem in file_list:
    total_file_list.append(elem)
# Se copian los archivos de la lista al directorio final (carpeta "test")
for file_addr in total_file_list:
  shutil.copyfile(file_addr, f"/content/output_sets/test/test/{os.path.basename(file_addr)}")
total_file_list.clear()

In [ ]:
# Se comprueba número de archivos en cada set
file_list = [x for x in glob.glob(f"/content/output_sets/val/test/*.jpg")]
print(len(file_list))

4999


In [ ]:
# Se procede a comprimir los sets
%cd /content/output_sets/test/test
!zip -r ../test.zip ./
%cd /content/output_sets/val/test
!zip -r ../test.zip ./

In [ ]:
# Se detiene ejecución automática
#assert False

In [ ]:
# Finalmente, se copian los sets generados a una carpeta en la unidad de GDrive
#!mkdir /mydrive/Generacion_datasets_eval_balanceados_v1/
#%cd /mydrive/Generacion_datasets_eval_balanceados_v1/
#!mkdir ./validacion/ ./testeo/
#!cp /content/output_sets/test/test.zip ./testeo/test.zip
#!cp /content/output_sets/val/test.zip ./validacion/test.zip

# Generación de Mosaicos

In [ ]:
# Se define la dirección de ambos conjuntos
test_set_addr = "/content/output_sets/test/test"
val_set_addr = "/content/output_sets/val/test"

In [ ]:
# Se crean carpetas
%cd /content/
!mkdir -v ./Custom_Mosaics/ ./Custom_Mosaics/test/ ./Custom_Mosaics/val/ # directorios de salida para mosaicos
!mkdir -v ./Ejemplos_negativos # directorio para los ejemplos negativos

In [ ]:
# Se descargan los ejemplos negativos:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1377C02YttAOMGNwyZjkK78M_2wPTGU8q' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1377C02YttAOMGNwyZjkK78M_2wPTGU8q" -O ./Ejemplos_negativos/obj.zip && rm -rf /tmp/cookies.txt

In [ ]:
# Se descomprime archivo .zip con ejemplos negativos
!unzip -q ./Ejemplos_negativos/obj.zip -d ./Ejemplos_negativos/
# Una vez descomprimidos los archivos, se borra el archivo .zip
!rm ./Ejemplos_negativos/obj.zip

In [ ]:
# Se aplica volveo horizontal a los ejemplos negativos
from PIL import Image, ImageOps
import PIL
import glob
img_list_ejemp = [x for x in glob.glob("/content/Ejemplos_negativos/*.jpg")]
contador = 0 # contador para evitar nombres repetidos
for neg_img in img_list_ejemp:
  this_img = Image.open(neg_img) # se abre imagen de ejemplo negativo
  this_img_mirror=ImageOps.mirror(this_img) # se aplica horizontal flip
  this_img_mirror.save(f"/content/Ejemplos_negativos/flip_{contador}.jpg") # se guarda imagen volteada
  this_img.close() # se cierra imagen
  this_img_mirror.close() # se cierra imagen con horizontal flip
  contador += 1 # se incrementa contador

## Conjunto de Validación

In [ ]:
# Se genera lista con nombres de archivos en conjunto de validación
img_list = list() # se inicializa lista para imágenes
# Se agregan los nombres de las imágenes a la lista
img_list = [x for x in glob.glob(f"{val_set_addr}/*.jpg")]
# Se imprime número de imágenes encontradas en conjunto de entrenamiento
print(f'[INFO]: Número de imágenes encontradas en set de validación: {len(img_list)}')
# Se reordena la lista con nombres de manera aleatoria
random.seed(123) # se genera semilla para obtener siempre el mismo resultado
random.shuffle(img_list) # se reordena la lista de manera aleatoria

In [ ]:
# Se genera lista con nombres de archivos en conjunto de ejemplos negativos
neg_img_list = list() # se inicializa lista de imágenes
# Se agergan los nombres de las imágenes a la lista
neg_img_list = [x for x in glob.glob(f"/content/Ejemplos_negativos/*.jpg")]
# Se imprime número de imágenes encontradas en conjunto de ejemplos negativos
print(f'[INFO]: Número de imágenes encontradas en set de ejemplos negativos: {len(neg_img_list)}')
# Se reordena la lista con nombres de manera aleatoria
random.seed(123) # se genera semilla para obtener siempre el mismo resultado
random.shuffle(neg_img_list) # se reordena la lista de manera aleatoria

In [ ]:
# Se define función auxiliar para detectar colisiones entre imágenes
from collections import namedtuple
Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')
def area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy

In [ ]:
# Se establece número de imágenes por mosaico
n_img_por_mosaico = 7 # este valor debe ser ingresado por el usuario
# Se establece resolución de imagen de salida
res_mosaico = [1024,768] # [width, height]
# Aplicar máscara?
do_Mask = True

In [ ]:
# Se va al directorio de salida
%cd /content/Custom_Mosaics/val/

In [ ]:
from IPython.core.display import Javascript
# Se definen tamaños máximos y mínimos para las imágenes de rostros
max_face_res = [90, 90] # [ancho, alto]
min_face_res = [25, 25] # [ancho, alto]
# Se calcula el número de imágenes para la proporción definida
n_L = 1
n_M = 1
n_S = 1
# Proporción del fondo por sección (S, M, L)
b_size_L = 0.35
b_size_M = 0.3
b_size_S = (1 - b_size_L - b_size_M)
# Se calcula el número de píxels verticales del fondo que corresponde a cada proporción
b_L = int(b_size_L*res_mosaico[1])
b_M = int(b_size_M*res_mosaico[1])
b_S = res_mosaico[1] - b_L - b_M

# Se inicializa lista con registro de coordenadas
img_reg = list()
neg_img_counter = 0 # contador de imágenes negativas
process_counter = 0 # contador de progreso

# Se inicializa variable contadora de imágenes generadas
j = 0
# Se comienza con la generación de los mosaicos
while j < len(img_list):
  # se crea archivo de texto para guardar anotaciones (etiqueta)
  background_label = open(f'{j}.txt', 'w')
  # Se crea fondo negro
  background = Image.new('RGB', res_mosaico, (0, 0, 0))
  # Se define número de ejemplos negativos a agregar por mosaico generado
  n_ejem_neg_por_mosaic = 10
  # Se inicializa lista con registro de vértices de imágenes ya colocadas
  img_reg = [(0, 0, 2, 2)] # cuadrado de lado 2
  # Se agregan los ejemplos negativos de manera aleatória
  for ji in range(n_ejem_neg_por_mosaic):
    # si el contador igualó o sobrepasó el máximo, se reinicia
    if ji+neg_img_counter >= len(neg_img_list): neg_img_counter = 0
    # Se abre imagen de ejemplo negativo a agregar
    ejem_neg = Image.open(f'{neg_img_list[ji + neg_img_counter]}')
    # Se genera propuesta de coordenadas aleatoriamente
    neg_pos_x = random.randint(0, res_mosaico[0])
    neg_pos_y = random.randint(0, b_S + b_M)
    # Se agrega el ejemplo negativo al fondo
    background.paste(ejem_neg, (neg_pos_x, neg_pos_y))
    # Se cierra imagen abierta con ejemplo negativo
    ejem_neg.close()
  # se agrega el número de ejemplos negatios agregados al contador
  neg_img_counter = neg_img_counter + n_ejem_neg_por_mosaic

  # Contador de colisiones
  n_colisiones = 0
  # Se verifica si se llegó a las últimas imágenes
  if (len(img_list) - j) < n_img_por_mosaico:
    n_img_por_mosaico = len(img_list) - j # se reduce número de rostros por imagen generada
  # Se agregan las fotos con rostros al fondo
  for i in range(n_img_por_mosaico):
    # Se abre imagen con rostro
    face = Image.open(f'{img_list[i+j]}')
    # se abre archivo de txt del rostro
    label_name = f"{img_list[i+j]}"
    face_label = open(f'{label_name[:-4]}.txt','r')
    parameters = face_label.readlines() # Se lee la línea con los parámetros del rostro
    parameters = parameters[0].split(" ") # Se separan los parámetros y se guardan en forma de lista
    # Se asigna el número de copias del mismo rostro dependiendo de la clase de la imagen
    if int(parameters[0])==0: #Neutral
      n_L = 2
      n_M = 1
      n_S = 1
    elif int(parameters[0])==1: #Happy
      n_L = 2
      n_M = 2
      n_S = 2
    elif int(parameters[0])==2: #Sad
      n_L = 1
      n_M = 1
      n_S = 1
    elif int(parameters[0])==3: #Surprise
      n_L = 1
      n_M = 1
      n_S = 0
    elif int(parameters[0])==4: #Fear
      n_L = 1
      n_M = 1
      n_S = 0
    elif int(parameters[0])==5: #Disgust
      n_L = 1
      n_M = 0
      n_S = 0
    elif int(parameters[0])==6: #Anger
      n_L = 1
      n_M = 1
      n_S = 1
    else: #Contempt
      n_L = 1
      n_M = 0
      n_S = 0
    
    # Se copian los rostros al mosaico en los distintos tamaños
    for ii in range(n_S + n_M + n_L):
      if ii < n_L:
        # número de imágenes por sección
        n_img = n_L
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = 2*int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        xface_max = max_face_res[0]
        yface_min = 2*int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        yface_max = max_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = (res_mosaico[1] - b_L)
        y_max = (res_mosaico[1] - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      elif ii >= n_L and ii < (n_M + n_L): 
        # número de imágenes por sección
        n_img = n_M
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        xface_max = 2*int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        yface_min = int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        yface_max = 2*int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = (res_mosaico[1] - b_L - b_M)
        y_max = (res_mosaico[1] - b_L - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      elif ii >= (n_M + n_L) and ii < (n_S + n_M + n_L):
        # número de imágenes por sección
        n_img = n_S
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = min_face_res[0]
        xface_max = int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        yface_min = min_face_res[1]
        yface_max = int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = 0
        y_max = (res_mosaico[1] - b_L - b_M - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      # Se calcula nueva dimensión de imagen con rostro
      new_x_size = random.randint(xface_min, xface_max)
      #new_y_size = random.randint(yface_min, yface_max) # se pierde aspect ratio :(
      #Se calcula el alto para conservar aspect ratio
      wpercent = (new_x_size/float(face.size[0]))
      new_y_size = int((float(face.size[1])*float(wpercent)))
      # Se escala la imagen seleccionada
      face = face.resize((new_x_size, new_y_size), resample=PIL.Image.BILINEAR)
      # Se calcula posición de la imagen asegurando la no intersección con imágenes anteriores
      do_intersect = True # variable de verificación
      # Se genera posición nueva y se verifica la no intersección
      while do_intersect:
        # generación de coordenadas nuevas
        new_x_pos = random.randint(x_min, x_max)
        new_y_pos = random.randint(y_min, y_max)
        # Se define vértice inferior derecho de la img
        v_inf_der = (new_x_pos, new_y_pos)
        # Se calcula el vértice superior izquierdo de la img
        v_sup_izq = (new_x_pos - new_x_size, new_y_pos - new_y_size)

        # Se define padding adicional a la imagen
        padding = 4 # píxeles

        # Se detecta intersección entre otras imágenes
        c2 = (v_sup_izq[0] - padding, v_sup_izq[1] - padding, v_inf_der[0] + padding , v_inf_der[1] + padding)
        d2 = Rectangle(c2[0],c2[1],c2[2],c2[3])
        for old_coord in img_reg:
          c1 = np.asarray(old_coord)
          d1 = Rectangle(c1[0],c1[1],c1[2],c1[3])
          #Se evalúa si la intersección entre ambas imágenes es distinto de vacío
          if area(d1,d2) is None:
            do_intersect = False # No se detectó intersección
          else:
            do_intersect = True # Se detectó intersección
            n_colisiones = n_colisiones + 1
            break # se detiene ciclo for y se vuelve a generar coordenadas

      # Una vez se verifició que no hay colisión, se guarda coordenadas en registro
      img_reg.append(c2)

      # Se agrega máscara circular si do_Mask es True
      if do_Mask:
        #Se aplica máscara circular
        lum_img = Image.new('L',[new_x_size,new_y_size] ,0) 
        draw = ImageDraw.Draw(lum_img)
        draw.pieslice([(0,0),(new_x_size,new_y_size)],0,360,fill=255)
        # Se pega la imagen en el fondo
        background.paste(face, (new_x_pos, new_y_pos),lum_img)

      # Se abre archivo de texto del respectivo rostro
      label_name = f"{img_list[i+j]}"
      face_label = open(f'{label_name[:-4]}.txt','r')
      parameters = face_label.readlines() # Se lee la línea con los parámetros del rostro
      parameters = parameters[0].split(" ") # Se separan los parámetros y se guardan en forma de lista
      parameters[1] = ((float(parameters[1])*new_x_size)+new_x_pos)/res_mosaico[0]
      parameters[2] = ((float(parameters[2])*new_y_size)+new_y_pos)/res_mosaico[1]
      parameters[3] = ((float(parameters[3])*new_x_size))/res_mosaico[0]
      parameters[4]  = ((float(parameters[4])*new_y_size))/res_mosaico[1]
      # Se genera string con parámetros y se escribe en etiqueta de imagen generada
      parameters_line = ' '.join(map(str,parameters))
      # Se escriben los parámetros en el archivo de texto
      background_label.write(f"{parameters_line}\n")
      face_label.close() # Se cierra el archivo

  # Se guarda imagen de fondo modificado
  background.save(f"{j}.jpg")

  # Se guarda archivo de texto
  background_label.close()

  # Se incrementa contador de imágenes con las imágenes procesadas
  j = j + n_img_por_mosaico
  # Se imprime mensaje de progreso
  print(f'Progreso: {float(j/len(img_list)*100):.2f}%', end = '', flush=True)
  #print("", end = '\r')
  print("")
  # Se incrementa el contador
  process_counter += 1
print("[INFO]: Conjunto de validación generado correctamente.")

## Conjunto de Testeo

In [ ]:
# Se genera lista con nombres de archivos en conjunto de validación
img_list = list() # se inicializa lista para imágenes
# Se agregan los nombres de las imágenes a la lista
img_list = [x for x in glob.glob(f"{test_set_addr}/*.jpg")]
# Se imprime número de imágenes encontradas en conjunto de entrenamiento
print(f'[INFO]: Número de imágenes encontradas en set de testeo: {len(img_list)}')
# Se reordena la lista con nombres de manera aleatoria
random.seed(123) # se genera semilla para obtener siempre el mismo resultado
random.shuffle(img_list) # se reordena la lista de manera aleatoria

In [ ]:
# Se genera lista con nombres de archivos en conjunto de ejemplos negativos
neg_img_list = list() # se inicializa lista de imágenes
# Se agergan los nombres de las imágenes a la lista
neg_img_list = [x for x in glob.glob(f"/content/Ejemplos_negativos/*.jpg")]
# Se imprime número de imágenes encontradas en conjunto de ejemplos negativos
print(f'[INFO]: Número de imágenes encontradas en set de ejemplos negativos: {len(neg_img_list)}')
# Se reordena la lista con nombres de manera aleatoria
random.seed(123) # se genera semilla para obtener siempre el mismo resultado
random.shuffle(neg_img_list) # se reordena la lista de manera aleatoria

In [ ]:
# Se establece número de imágenes por mosaico
n_img_por_mosaico = 7 # este valor debe ser ingresado por el usuario
# Se establece resolución de imagen de salida
res_mosaico = [1024,768] # [width, height]
# Aplicar máscara?
do_Mask = True

In [ ]:
# Se va al directorio de salida
%cd /content/Custom_Mosaics/test/

In [ ]:
from IPython.core.display import Javascript
# Se definen tamaños máximos y mínimos para las imágenes de rostros
max_face_res = [90, 90] # [ancho, alto]
min_face_res = [25, 25] # [ancho, alto]
# Se calcula el número de imágenes para la proporción definida
n_L = 1
n_M = 1
n_S = 1
# Proporción del fondo por sección (S, M, L)
b_size_L = 0.4
b_size_M = 0.25
b_size_S = (1 - b_size_L - b_size_M)
# Se calcula el número de píxels verticales del fondo que corresponde a cada proporción
b_L = int(b_size_L*res_mosaico[1])
b_M = int(b_size_M*res_mosaico[1])
b_S = res_mosaico[1] - b_L - b_M

# Se inicializa lista con registro de coordenadas
img_reg = list()
neg_img_counter = 0 # contador de imágenes negativas
process_counter = 0 # contador de progreso

# Se inicializa variable contadora de imágenes generadas
j = 0
# Se comienza con la generación de los mosaicos
while j < len(img_list):
  # se crea archivo de texto para guardar anotaciones (etiqueta)
  background_label = open(f'{j}.txt', 'w')
  # Se crea fondo negro
  background = Image.new('RGB', res_mosaico, (0, 0, 0))
  # Se define número de ejemplos negativos a agregar por mosaico generado
  n_ejem_neg_por_mosaic = 10
  # Se inicializa lista con registro de vértices de imágenes ya colocadas
  img_reg = [(0, 0, 2, 2)] # cuadrado de lado 2
  # Se agregan los ejemplos negativos de manera aleatória
  for ji in range(n_ejem_neg_por_mosaic):
    # si el contador igualó o sobrepasó el máximo, se reinicia
    if ji+neg_img_counter >= len(neg_img_list): neg_img_counter = 0
    # Se abre imagen de ejemplo negativo a agregar
    ejem_neg = Image.open(f'{neg_img_list[ji + neg_img_counter]}')
    # Se genera propuesta de coordenadas aleatoriamente
    neg_pos_x = random.randint(0, res_mosaico[0])
    neg_pos_y = random.randint(0, b_S + b_M)
    # Se agrega el ejemplo negativo al fondo
    background.paste(ejem_neg, (neg_pos_x, neg_pos_y))
    # Se cierra imagen abierta con ejemplo negativo
    ejem_neg.close()
  # se agrega el número de ejemplos negatios agregados al contador
  neg_img_counter = neg_img_counter + n_ejem_neg_por_mosaic

  # Contador de colisiones
  n_colisiones = 0
  # Se verifica si se llegó a las últimas imágenes
  if (len(img_list) - j) < n_img_por_mosaico:
    n_img_por_mosaico = len(img_list) - j # se reduce número de rostros por imagen generada
  # Se agregan las fotos con rostros al fondo
  for i in range(n_img_por_mosaico):
    # Se abre imagen con rostro
    face = Image.open(f'{img_list[i+j]}')
    # se abre archivo de txt del rostro
    label_name = f"{img_list[i+j]}"
    face_label = open(f'{label_name[:-4]}.txt','r')
    parameters = face_label.readlines() # Se lee la línea con los parámetros del rostro
    parameters = parameters[0].split(" ") # Se separan los parámetros y se guardan en forma de lista
    # Se asigna el número de copias del mismo rostro dependiendo de la clase de la imagen
    if int(parameters[0])==0: #Neutral
      n_L = 2
      n_M = 1
      n_S = 1
    elif int(parameters[0])==1: #Happy
      n_L = 2
      n_M = 2
      n_S = 2
    elif int(parameters[0])==2: #Sad
      n_L = 1
      n_M = 1
      n_S = 1
    elif int(parameters[0])==3: #Surprise
      n_L = 1
      n_M = 1
      n_S = 0
    elif int(parameters[0])==4: #Fear
      n_L = 1
      n_M = 1
      n_S = 0
    elif int(parameters[0])==5: #Disgust
      n_L = 1
      n_M = 0
      n_S = 0
    elif int(parameters[0])==6: #Anger
      n_L = 1
      n_M = 1
      n_S = 1
    else: #Contempt
      n_L = 1
      n_M = 0
      n_S = 0
    
    # Se copian los rostros al mosaico en los distintos tamaños
    for ii in range(n_S + n_M + n_L):
      if ii < n_L:
        # número de imágenes por sección
        n_img = n_L
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = 2*int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        xface_max = max_face_res[0]
        yface_min = 2*int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        yface_max = max_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = (res_mosaico[1] - b_L)
        y_max = (res_mosaico[1] - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      elif ii >= n_L and ii < (n_M + n_L): 
        # número de imágenes por sección
        n_img = n_M
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        xface_max = 2*int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        yface_min = int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        yface_max = 2*int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = (res_mosaico[1] - b_L - b_M)
        y_max = (res_mosaico[1] - b_L - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      elif ii >= (n_M + n_L) and ii < (n_S + n_M + n_L):
        # número de imágenes por sección
        n_img = n_S
        # tamaño máximo y mínimo para cada rostro reescalado
        xface_min = min_face_res[0]
        xface_max = int((max_face_res[0] - min_face_res[0])/3) + min_face_res[0]
        yface_min = min_face_res[1]
        yface_max = int((max_face_res[1] - min_face_res[1])/3) + min_face_res[1]
        # posición vertical mínima y máxima admitida
        y_min = 0
        y_max = (res_mosaico[1] - b_L - b_M - yface_max)
        # posición horizontal mínima y máxima admitida
        x_min = 0
        x_max = res_mosaico[0]-xface_max

      # Se calcula nueva dimensión de imagen con rostro
      new_x_size = random.randint(xface_min, xface_max)
      #new_y_size = random.randint(yface_min, yface_max) # se pierde aspect ratio :(
      #Se calcula el alto para conservar aspect ratio
      wpercent = (new_x_size/float(face.size[0]))
      new_y_size = int((float(face.size[1])*float(wpercent)))
      # Se escala la imagen seleccionada
      face = face.resize((new_x_size, new_y_size), resample=PIL.Image.BILINEAR)
      # Se calcula posición de la imagen asegurando la no intersección con imágenes anteriores
      do_intersect = True # variable de verificación
      # Se genera posición nueva y se verifica la no intersección
      while do_intersect:
        # generación de coordenadas nuevas
        new_x_pos = random.randint(x_min, x_max)
        new_y_pos = random.randint(y_min, y_max)
        # Se define vértice inferior derecho de la img
        v_inf_der = (new_x_pos, new_y_pos)
        # Se calcula el vértice superior izquierdo de la img
        v_sup_izq = (new_x_pos - new_x_size, new_y_pos - new_y_size)

        # Se define padding adicional a la imagen
        padding = 4 # píxeles

        # Se detecta intersección entre otras imágenes
        c2 = (v_sup_izq[0] - padding, v_sup_izq[1] - padding, v_inf_der[0] + padding , v_inf_der[1] + padding)
        d2 = Rectangle(c2[0],c2[1],c2[2],c2[3])
        for old_coord in img_reg:
          c1 = np.asarray(old_coord)
          d1 = Rectangle(c1[0],c1[1],c1[2],c1[3])
          #Se evalúa si la intersección entre ambas imágenes es distinto de vacío
          if area(d1,d2) is None:
            do_intersect = False # No se detectó intersección
          else:
            do_intersect = True # Se detectó intersección
            n_colisiones = n_colisiones + 1
            break # se detiene ciclo for y se vuelve a generar coordenadas

      # Una vez se verifició que no hay colisión, se guarda coordenadas en registro
      img_reg.append(c2)

      # Se agrega máscara circular si do_Mask es True
      if do_Mask:
        #Se aplica máscara circular
        lum_img = Image.new('L',[new_x_size,new_y_size] ,0) 
        draw = ImageDraw.Draw(lum_img)
        draw.pieslice([(0,0),(new_x_size,new_y_size)],0,360,fill=255)
        # Se pega la imagen en el fondo
        background.paste(face, (new_x_pos, new_y_pos),lum_img)

      # Se abre archivo de texto del respectivo rostro
      label_name = f"{img_list[i+j]}"
      face_label = open(f'{label_name[:-4]}.txt','r')
      parameters = face_label.readlines() # Se lee la línea con los parámetros del rostro
      parameters = parameters[0].split(" ") # Se separan los parámetros y se guardan en forma de lista
      parameters[1] = ((float(parameters[1])*new_x_size)+new_x_pos)/res_mosaico[0]
      parameters[2] = ((float(parameters[2])*new_y_size)+new_y_pos)/res_mosaico[1]
      parameters[3] = ((float(parameters[3])*new_x_size))/res_mosaico[0]
      parameters[4]  = ((float(parameters[4])*new_y_size))/res_mosaico[1]
      # Se genera string con parámetros y se escribe en etiqueta de imagen generada
      parameters_line = ' '.join(map(str,parameters))
      # Se escriben los parámetros en el archivo de texto
      background_label.write(f"{parameters_line}\n")
      face_label.close() # Se cierra el archivo

  # Se guarda imagen de fondo modificado
  background.save(f"{j}.jpg")

  # Se guarda archivo de texto
  background_label.close()

  # Se incrementa contador de imágenes con las imágenes procesadas
  j = j + n_img_por_mosaico
  # Se imprime mensaje de progreso
  print(f'Progreso: {float(j/len(img_list)*100):.2f}%', end = '', flush=True)
  #print("", end = '\r')
  print("")
  # Se incrementa el contador
  process_counter += 1
print("[INFO]: Conjunto de validación generado correctamente.")

## Guardado de los mosaicos

In [ ]:
# se verifica número de imágenes en ambos conjuntos generados
# Validación
img_list = [x for x in glob.glob("/content/Custom_Mosaics/val/*.jpg")]
print(f"Total de mosaicos en conjunto de validación: {len(img_list)}")
# Testeo
img_list = [x for x in glob.glob("/content/Custom_Mosaics/test/*.jpg")]
print(f"Total de mosaicos en conjunto de testeo: {len(img_list)}")

In [ ]:
# Se crean directorios para guardado
!mkdir /mydrive/Generacion_datasets_eval_balanceados_v1/Custom_Mosaics
!mkdir /mydrive/Generacion_datasets_eval_balanceados_v1/Custom_Mosaics/testeo
!mkdir /mydrive/Generacion_datasets_eval_balanceados_v1/Custom_Mosaics/validacion

In [ ]:
# Se procede a comprimir y copiar los datasets a GDrive
%cd /content/Custom_Mosaics/testeo/
!zip -r /mydrive/Generacion_datasets_eval_balanceados_v1/Custom_Mosaics/testeo/test.zip ./
%cd /content/Custom_Mosaics/validacion/
!zip -r /mydrive/Generacion_datasets_eval_balanceados_v1/Custom_Mosaics/validacion/test.zip ./